In [1]:
import re
import pandas as pd
from itertools import product

def results_text_to_clipboard_df(text: str):
    """
    Parse lines like:
      [train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.29588704978190755
    Build a DF with columns: event, 5_1, 5_2, 5_3, 10_1, ... 50_3
    Copy DF to clipboard for easy paste into Sheets.
    """
    # event, lb size, set number, score
    rx = re.compile(
        r"\[train\[(.+?)_lb(\d+)_set(\d+)\]\]\s*macro-F1\s*=\s*([0-9]*\.?[0-9]+)",
        flags=re.IGNORECASE
    )

    rows = []
    for m in rx.finditer(text):
        event, lb, setnum, score = m.group(1), m.group(2), m.group(3), m.group(4)
        rows.append((event, int(lb), int(setnum), float(score)))

    if not rows:
        print("No matches found. Check the text format or regex.")
        return None

    # Build wide table
    df_long = pd.DataFrame(rows, columns=["event", "lb", "set", "score"])
    # Keep only the requested lb/set grid
    wanted_lbs  = [5, 10, 25, 50]
    wanted_sets = [1, 2, 3]
    df_long = df_long[df_long["lb"].isin(wanted_lbs) & df_long["set"].isin(wanted_sets)]

    # Pivot to event rows, columns like "5_1"
    df_wide = df_long.assign(col=lambda d: d["lb"].astype(str) + "_" + d["set"].astype(str)) \
                     .pivot_table(index="event", columns="col", values="score", aggfunc="first")

    # Ensure full cartesian column order exists
    all_cols = [f"{lb}_{s}" for lb, s in product(wanted_lbs, wanted_sets)]
    df_wide = df_wide.reindex(columns=all_cols)

    # Put 'event' as an actual column (since you asked for it in the output)
    df_out = df_wide.reset_index()

    # Copy to clipboard
    # df_out.to_clipboard(index=False)
    print("✅ Copied to clipboard. Preview:")
    print(df_out)
    return df_out

# ---- Example usage ----
text = """[train[hurricane_harvey_2017_lb5_set1]] macro-F1 = 0.22034339966948824
[train[hurricane_harvey_2017_lb5_set2]] macro-F1 = 0.14569320968964009
[train[hurricane_harvey_2017_lb5_set3]] macro-F1 = 0.3393924852013599
[train[cyclone_idai_2019_lb5_set1]] macro-F1 = 0.3501891693108702
[train[cyclone_idai_2019_lb5_set2]] macro-F1 = 0.21153819393164902
[train[cyclone_idai_2019_lb5_set3]] macro-F1 = 0.3630123186525299
[train[kaikoura_earthquake_2016_lb5_set1]] macro-F1 = 0.42801950447513654
[train[kaikoura_earthquake_2016_lb5_set2]] macro-F1 = 0.39468223181920653
[train[kaikoura_earthquake_2016_lb5_set3]] macro-F1 = 0.508425876022931
[train[hurricane_irma_2017_lb5_set1]] macro-F1 = 0.2533785906990818
[train[hurricane_irma_2017_lb5_set2]] macro-F1 = 0.2786831017138125
[train[hurricane_irma_2017_lb5_set3]] macro-F1 = 0.28000018150812084
[train[kerala_floods_2018_lb5_set1]] macro-F1 = 0.06921693643110247
[train[kerala_floods_2018_lb5_set2]] macro-F1 = 0.19620711849335123
[train[kerala_floods_2018_lb5_set3]] macro-F1 = 0.24364812539796107
[train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.4119668498516313
[train[canada_wildfires_2016_lb5_set2]] macro-F1 = 0.38360381498703633
[train[canada_wildfires_2016_lb5_set3]] macro-F1 = 0.11563441390666167
[train[hurricane_dorian_2019_lb5_set1]] macro-F1 = 0.3451727228154396
[train[hurricane_dorian_2019_lb5_set2]] macro-F1 = 0.23618361704402582
[train[hurricane_dorian_2019_lb5_set3]] macro-F1 = 0.3531805521187237
[train[hurricane_maria_2017_lb5_set1]] macro-F1 = 0.36169711049693576
[train[hurricane_maria_2017_lb5_set2]] macro-F1 = 0.23431387943655022
[train[hurricane_maria_2017_lb5_set3]] macro-F1 = 0.3892314112902348
[train[hurricane_florence_2018_lb5_set1]] macro-F1 = 0.3146520563605324
[train[hurricane_florence_2018_lb5_set2]] macro-F1 = 0.35053067850683395
[train[hurricane_florence_2018_lb5_set3]] macro-F1 = 0.4738578069570385
[train[california_wildfires_2018_lb5_set1]] macro-F1 = 0.32176194872546277
[train[california_wildfires_2018_lb5_set2]] macro-F1 = 0.3411246820112449
[train[california_wildfires_2018_lb5_set3]] macro-F1 = 0.13566195629705044
[train[hurricane_harvey_2017_lb10_set1]] macro-F1 = 0.5833171016852385
[train[hurricane_harvey_2017_lb10_set2]] macro-F1 = 0.45476327090757235
[train[hurricane_harvey_2017_lb10_set3]] macro-F1 = 0.5326199637902242
[train[cyclone_idai_2019_lb10_set1]] macro-F1 = 0.46625588183253786
[train[cyclone_idai_2019_lb10_set2]] macro-F1 = 0.5035315738819376
[train[cyclone_idai_2019_lb10_set3]] macro-F1 = 0.0069392812887236685
[train[kaikoura_earthquake_2016_lb10_set1]] macro-F1 = 0.6579205147447788
[train[kaikoura_earthquake_2016_lb10_set2]] macro-F1 = 0.5944623868856994
[train[kaikoura_earthquake_2016_lb10_set3]] macro-F1 = 0.6127846396193644
[train[hurricane_irma_2017_lb10_set1]] macro-F1 = 0.49738367141603335
[train[hurricane_irma_2017_lb10_set2]] macro-F1 = 0.4679677657940624
[train[hurricane_irma_2017_lb10_set3]] macro-F1 = 0.4935259679703406
[train[kerala_floods_2018_lb10_set1]] macro-F1 = 0.30049203048914375
[train[kerala_floods_2018_lb10_set2]] macro-F1 = 0.33702257533129204
[train[kerala_floods_2018_lb10_set3]] macro-F1 = 0.4517646485362786
[train[canada_wildfires_2016_lb10_set1]] macro-F1 = 0.49098562386853123
[train[canada_wildfires_2016_lb10_set2]] macro-F1 = 0.548548662803747
[train[canada_wildfires_2016_lb10_set3]] macro-F1 = 0.4392577880183098
[train[hurricane_dorian_2019_lb10_set1]] macro-F1 = 0.49210931807004954
[train[hurricane_dorian_2019_lb10_set2]] macro-F1 = 0.3961785315299573
[train[hurricane_dorian_2019_lb10_set3]] macro-F1 = 0.44529243745364055
[train[hurricane_maria_2017_lb10_set1]] macro-F1 = 0.5235771384029844
[train[hurricane_maria_2017_lb10_set2]] macro-F1 = 0.5552163479823324
[train[hurricane_maria_2017_lb10_set3]] macro-F1 = 0.5167766962404641
[train[hurricane_florence_2018_lb10_set1]] macro-F1 = 0.4686436649828117
[train[hurricane_florence_2018_lb10_set2]] macro-F1 = 0.4217271714617469
[train[hurricane_florence_2018_lb10_set3]] macro-F1 = 0.5507139152122852
[train[california_wildfires_2018_lb10_set1]] macro-F1 = 0.38890360529920764
[train[california_wildfires_2018_lb10_set2]] macro-F1 = 0.41103070325459556
[train[california_wildfires_2018_lb10_set3]] macro-F1 = 0.3532741781525816
[train[hurricane_harvey_2017_lb25_set1]] macro-F1 = 0.5737937025542823
[train[hurricane_harvey_2017_lb25_set2]] macro-F1 = 0.616580957863315
[train[hurricane_harvey_2017_lb25_set3]] macro-F1 = 0.5663279077330405
[train[cyclone_idai_2019_lb25_set1]] macro-F1 = 0.5573353729493362
[train[cyclone_idai_2019_lb25_set2]] macro-F1 = 0.5502966341191173
[train[cyclone_idai_2019_lb25_set3]] macro-F1 = 0.563868646190371
[train[kaikoura_earthquake_2016_lb25_set1]] macro-F1 = 0.6764512528783891
[train[kaikoura_earthquake_2016_lb25_set2]] macro-F1 = 0.7134899882458959
[train[kaikoura_earthquake_2016_lb25_set3]] macro-F1 = 0.6812765814735431
[train[hurricane_irma_2017_lb25_set1]] macro-F1 = 0.5894547811623793
[train[hurricane_irma_2017_lb25_set2]] macro-F1 = 0.6038350030415943
[train[hurricane_irma_2017_lb25_set3]] macro-F1 = 0.5791417631318799
[train[kerala_floods_2018_lb25_set1]] macro-F1 = 0.46704920730919713
[train[kerala_floods_2018_lb25_set2]] macro-F1 = 0.47979793133610166
[train[kerala_floods_2018_lb25_set3]] macro-F1 = 0.5081128621246243
[train[canada_wildfires_2016_lb25_set1]] macro-F1 = 0.5221160128029627
[train[canada_wildfires_2016_lb25_set2]] macro-F1 = 0.5761565208695866
[train[canada_wildfires_2016_lb25_set3]] macro-F1 = 0.5722950884189537
[train[hurricane_dorian_2019_lb25_set1]] macro-F1 = 0.5697188687112722
[train[hurricane_dorian_2019_lb25_set2]] macro-F1 = 0.5500019307338977
[train[hurricane_dorian_2019_lb25_set3]] macro-F1 = 0.5837469195169039
[train[hurricane_maria_2017_lb25_set1]] macro-F1 = 0.59844662588489
[train[hurricane_maria_2017_lb25_set2]] macro-F1 = 0.5729247906658566
[train[hurricane_maria_2017_lb25_set3]] macro-F1 = 0.6195730345750454
[train[hurricane_florence_2018_lb25_set1]] macro-F1 = 0.6117956458952554
[train[hurricane_florence_2018_lb25_set2]] macro-F1 = 0.6408933452110481
[train[hurricane_florence_2018_lb25_set3]] macro-F1 = 0.5452864710196779
[train[california_wildfires_2018_lb25_set1]] macro-F1 = 0.6018375073434458
[train[california_wildfires_2018_lb25_set2]] macro-F1 = 0.5294501750534366
[train[california_wildfires_2018_lb25_set3]] macro-F1 = 0.4899469216586757
[train[hurricane_harvey_2017_lb50_set1]] macro-F1 = 0.6725676001233976
[train[hurricane_harvey_2017_lb50_set2]] macro-F1 = 0.642399639630128
[train[hurricane_harvey_2017_lb50_set3]] macro-F1 = 0.6514852036969988
[train[cyclone_idai_2019_lb50_set1]] macro-F1 = 0.5819439416294824
[train[cyclone_idai_2019_lb50_set2]] macro-F1 = 0.6107488257434519
[train[cyclone_idai_2019_lb50_set3]] macro-F1 = 0.5634844828176191
[train[kaikoura_earthquake_2016_lb50_set1]] macro-F1 = 0.7610126630561201
[train[kaikoura_earthquake_2016_lb50_set2]] macro-F1 = 0.7516959997289185
[train[kaikoura_earthquake_2016_lb50_set3]] macro-F1 = 0.6319284662956589
[train[hurricane_irma_2017_lb50_set1]] macro-F1 = 0.012608353033884948
[train[hurricane_irma_2017_lb50_set2]] macro-F1 = 0.628322694337138
[train[hurricane_irma_2017_lb50_set3]] macro-F1 = 0.5664087462813823
[train[kerala_floods_2018_lb50_set1]] macro-F1 = 0.5280665595578697
[train[kerala_floods_2018_lb50_set2]] macro-F1 = 0.5211518759486873
[train[kerala_floods_2018_lb50_set3]] macro-F1 = 0.553665236277239
[train[canada_wildfires_2016_lb50_set1]] macro-F1 = 0.5990660151376592
[train[canada_wildfires_2016_lb50_set2]] macro-F1 = 0.6245262973584771
[train[canada_wildfires_2016_lb50_set3]] macro-F1 = 0.5714120593007024
[train[hurricane_dorian_2019_lb50_set1]] macro-F1 = 0.5887618938759691
[train[hurricane_dorian_2019_lb50_set2]] macro-F1 = 0.6135842616452596
[train[hurricane_dorian_2019_lb50_set3]] macro-F1 = 0.616779773730435
[train[hurricane_maria_2017_lb50_set1]] macro-F1 = 0.6789994262250498
[train[hurricane_maria_2017_lb50_set2]] macro-F1 = 0.6632198822110538
[train[hurricane_maria_2017_lb50_set3]] macro-F1 = 0.6538802831209027
[train[hurricane_florence_2018_lb50_set1]] macro-F1 = 0.6943187166639099
[train[hurricane_florence_2018_lb50_set2]] macro-F1 = 0.6687238751648468
[train[hurricane_florence_2018_lb50_set3]] macro-F1 = 0.621255926304678
[train[california_wildfires_2018_lb50_set1]] macro-F1 = 0.6450983601978229
[train[california_wildfires_2018_lb50_set2]] macro-F1 = 0.6437752160571786
[train[california_wildfires_2018_lb50_set3]] macro-F1 = 0.591791808394935"""
results_text_to_clipboard_df(text)


✅ Copied to clipboard. Preview:
col                      event       5_1       5_2       5_3      10_1  \
0    california_wildfires_2018  0.321762  0.341125  0.135662  0.388904   
1        canada_wildfires_2016  0.411967  0.383604  0.115634  0.490986   
2            cyclone_idai_2019  0.350189  0.211538  0.363012  0.466256   
3        hurricane_dorian_2019  0.345173  0.236184  0.353181  0.492109   
4      hurricane_florence_2018  0.314652  0.350531  0.473858  0.468644   
5        hurricane_harvey_2017  0.220343  0.145693  0.339392  0.583317   
6          hurricane_irma_2017  0.253379  0.278683  0.280000  0.497384   
7         hurricane_maria_2017  0.361697  0.234314  0.389231  0.523577   
8     kaikoura_earthquake_2016  0.428020  0.394682  0.508426  0.657921   
9           kerala_floods_2018  0.069217  0.196207  0.243648  0.300492   

col      10_2      10_3      25_1      25_2      25_3      50_1      50_2  \
0    0.411031  0.353274  0.601838  0.529450  0.489947  0.645098  0.643775   

col,event,5_1,5_2,5_3,10_1,10_2,10_3,25_1,25_2,25_3,50_1,50_2,50_3
0,california_wildfires_2018,0.321762,0.341125,0.135662,0.388904,0.411031,0.353274,0.601838,0.529450,0.489947,0.645098,0.643775,0.591792
1,canada_wildfires_2016,0.411967,0.383604,0.115634,0.490986,0.548549,0.439258,0.522116,0.576157,0.572295,0.599066,0.624526,0.571412
2,cyclone_idai_2019,0.350189,0.211538,0.363012,0.466256,0.503532,0.006939,0.557335,0.550297,0.563869,0.581944,0.610749,0.563484
3,hurricane_dorian_2019,0.345173,0.236184,0.353181,0.492109,0.396179,0.445292,0.569719,0.550002,0.583747,0.588762,0.613584,0.616780
4,hurricane_florence_2018,0.314652,0.350531,0.473858,0.468644,0.421727,0.550714,0.611796,0.640893,0.545286,0.694319,0.668724,0.621256
5,hurricane_harvey_2017,0.220343,0.145693,0.339392,0.583317,0.454763,0.532620,0.573794,0.616581,0.566328,0.672568,0.642400,0.651485
6,hurricane_irma_2017,0.253379,0.278683,0.280000,0.497384,0.467968,0.493526,0.589455,0.603835,0.579142,0.012608,0.628323,0.566409
7,hurricane_maria_2017,0.361697,0.234314,0.389231,0.523577,0.555216,0.516777,0.598447,0.572925,0.619573,0.678999,0.663220,0.653880
8,kaikoura_earthquake_2016,0.428020,0.394682,0.508426,0.657921,0.594462,0.612785,0.676451,0.713490,0.681277,0.761013,0.751696,0.631928
9,kerala_floods_2018,0.069217,0.196207,0.243648,0.300492,0.337023,0.451765,0.467049,0.479798,0.508113,0.528067,0.521152,0.553665
